In [1]:
# requirements
from pymongo import MongoClient

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA

import joblib

import tqdm
import heapq
import json

In [2]:
# MongoDB 연결
client = MongoClient("mongodb://localhost:3999/")

In [3]:
# DB 불러오기
matchup_statistics_db = client["matchup_statistics_db"]  # summoners, indicators, matches

In [6]:
division = {"I": 1, "II": 2, "III": 3, "IV": 4}

users = []  # 아이언
my_rank = 1

# or (summoner["league"]["tier"] == "DIAMOND" and summoner["league"]["rank"] == "IV")

for summoner in matchup_statistics_db["summoners"].find():
    if (summoner["league"]["tier"] == "IRON" or summoner["league"]["tier"] == "BRONZE" or summoner["league"]["tier"] == "SILVER"):
        users.append(summoner["_id"])

print(len(users))
print(users[0])

13212
Ar_KKTRQ4khe-Lm8iDZzy5GLzag_4QfvTCZyXkQYodvTVBmx


In [ ]:
# 학습 데이터 추출

# collection 불러오기
indicators = matchup_statistics_db["indicators"].find({"_id": {"$in": users}})

# 학습 데이터
x = []
y = []

summoners = []

times = 0
all_zero = 0


# 결측치 처리 - 추후에는 평균 값으로 입력
def imputation(datas, indicator, record, record_detail):
    global times

    if datas[indicator][record].get(record_detail):
        return datas[indicator][record].get(record_detail)
    else:
        times += 1
        return 0

# 데이터 추출
for indicator in indicators:
    for match_indicator in indicator["matchIndicators"]:
        summoner = {}

        summoner["csDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["csDiffer"]
        summoner["expDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["expDiffer"]
        summoner["turretPlateDestroyDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["turretPlateDestroyDiffer"]
        summoner["dealDiffer"] = imputation(match_indicator, "laneIndicator", "aggresiveLaneAbilility", "dealDiffer")

        summoner["turretKillsPerDeaths"] = match_indicator["macroIndicator"]["splitPoint"]["turretKillsPerDeaths"]
        summoner["damageDealtToTurretsPerTeamTotalTowerDamageDone"] = match_indicator["macroIndicator"]["splitPoint"]["damageDealtToTurretsPerTeamTotalTowerDamageDone"]

        summoner["totalTimeCCingOthersPerTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["totalTimeCCingOthersPerTotalDamageTaken"]
        summoner["totalDamageTakenPerTeamTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["totalDamageTakenPerTeamTotalDamageTaken"]
        summoner["damageSelfMitigatedPerTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["damageSelfMitigatedPerTotalDamageTaken"]

        summoner["visionScorePerDeath"] = match_indicator["macroIndicator"]["visionPoint"]["visionScorePerDeath"]
        summoner["totalJungleObjectivePerGameDuration"] = match_indicator["macroIndicator"]["jungleHoldPoint"]["totalJungleObjectivePerGameDuration"]

        summoner["getObjectiveDifferPerGameDuration"] = match_indicator["macroIndicator"]["objectivePoint"]["getObjectiveDifferPerGameDuration"]

        summoner["damagePerMinute"] = match_indicator["macroIndicator"]["totalDealPoint"]["damagePerMinute"]
        summoner["dealPerGold"] = imputation(match_indicator, "macroIndicator", "totalDealPoint", "dealPerGold")
        summoner["teamDamagePercentage"] = imputation(match_indicator, "macroIndicator", "totalDealPoint", "teamDamagePercentage")

        if match_indicator["metadata"]["laneInfo"].get("teamPosition") == "TOP":
            pass


        is_all_zero = True
        idx = 0

        for key, value in summoner.items():
            if value != 0:
                is_all_zero = False
                break 
                
        if is_all_zero is False:

            if match_indicator["metadata"]["isWin"] is True:
                y.append(1)
            else:
                y.append(0)

            summoners.append(summoner)